In [2]:
import pandas as pd

# Load the SMS Spam Collection dataset
messages = pd.read_csv("/content/SMSSpamCollection.txt", sep='\t', header=None, names=["label", "message"])


In [3]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

stemmer = nltk.SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
corpus = []
def data_cleaning_fun(messages):
    for i in range(len(messages)):
      review = re.sub("[^a-zA-z]"," ",messages["message"][i])
      review = review.lower()
      review = review.split()
      review = [stemmer.stem(word) for word in review if word not in set(stopwords.words("english"))]
      review = " ".join(review)
      corpus.append(review)
    return corpus

data_cleaning_fun(messages)

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkts st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chgs send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breath

In [7]:
y = pd.get_dummies(messages["label"],dtype=int)
y

,ham,spam
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
5567,0,1
5568,1,0
5569,1,0
5570,1,0


In [8]:
y = y.iloc[:,1]
y

,spam
0,0
1,0
2,1
3,0
4,0
...,...
5567,1
5568,0
5569,0
5570,0


In [9]:
y.shape

(5572,)

In [10]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus,y,test_size = 20)

cv = CountVectorizer(max_features=2500,ngram_range=(1,2))
X_train = cv.fit_transform(X_train).toarray()
X_test = cv.transform(X_test).toarray()
print("Shape of X = ",X_train.shape)
print("Vocubulary = ",cv.vocabulary_)

Shape of X =  (5552, 2500)
Vocubulary =  {'smile': np.int64(1904), 'pleasur': np.int64(1605), 'pain': np.int64(1530), 'troubl': np.int64(2172), 'like': np.int64(1172), 'rain': np.int64(1692), 'sum': np.int64(2026), 'hurt': np.int64(968), 'becoz': np.int64(164), 'someon': np.int64(1918), 'still': np.int64(1980), 'love': np.int64(1219), 'see': np.int64(1821), 'hi': np.int64(911), 'call': np.int64(246), 'receiv': np.int64(1717), 'prize': np.int64(1668), 'draw': np.int64(577), 'pleas': np.int64(1600), 'claim': np.int64(353), 'transfer': np.int64(2163), 'els': np.int64(608), 'final': np.int64(695), 'came': np.int64(278), 'fix': np.int64(707), 'think': np.int64(2098), 'far': np.int64(677), 'find': np.int64(697), 'check': np.int64(340), 'googl': np.int64(836), 'place': np.int64(1591), 'yeah': np.int64(2473), 'bare': np.int64(149), 'enough': np.int64(623), 'room': np.int64(1772), 'two': np.int64(2187), 'us': np.int64(2282), 'mani': np.int64(1258), 'fuck': np.int64(751), 'sorri': np.int64(1930)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

spam_ham_detection_model_bow = GaussianNB()
spam_ham_detection_model_bow.fit(X_train,y_train)
spam_ham_detection_model_bow

GaussianNB()

In [12]:
y_pred = spam_ham_detection_model_bow.predict(X_test)
y_pred

array([0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0])

In [13]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_test,y_pred)*100

70.0

In [14]:
classification_report(y_test,y_pred)

'              precision    recall  f1-score   support\n\n           0       0.90      0.64      0.75        14\n           1       0.50      0.83      0.62         6\n\n    accuracy                           0.70        20\n   macro avg       0.70      0.74      0.69        20\nweighted avg       0.78      0.70      0.71        20\n'

In [17]:
# USING TF-IDF Model
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

## Applying train test split
X_train,X_test,y_train,y_test = train_test_split(corpus,y,test_size=20,random_state=42)

cv_tfidf = TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X_train = cv_tfidf.fit_transform(X_train).toarray()
X_test = cv_tfidf.transform(X_test).toarray()
print("Shape of X = ",X_train.shape)
print("Vocubulary = ",cv_tfidf.vocabulary_)

Shape of X =  (5552, 2500)
Vocubulary =  {'yeah': np.int64(2469), 'leav': np.int64(1151), 'coupl': np.int64(451), 'minut': np.int64(1308), 'amp': np.int64(65), 'let': np.int64(1162), 'know': np.int64(1100), 'get': np.int64(798), 'mu': np.int64(1372), 'yeah leav': np.int64(2471), 'let know': np.int64(1163), 'custom': np.int64(478), 'place': np.int64(1592), 'call': np.int64(248), 'cud': np.int64(470), 'go': np.int64(822), 'pm': np.int64(1619), 'got': np.int64(861), 'way': np.int64(2329), 'contact': np.int64(425), 'guess': np.int64(894), 'worri': np.int64(2419), 'must': np.int64(1380), 'bodi': np.int64(196), 'quit': np.int64(1686), 'sure': np.int64(2039), 'take': np.int64(2057), 'slow': np.int64(1898), 'first': np.int64(713), 'test': np.int64(2085), 'guid': np.int64(895), 'relax': np.int64(1729), 'noth': np.int64(1446), 'said': np.int64(1787), 'reason': np.int64(1713), 'keep': np.int64(1080), 'nd': np.int64(1391), 'time': np.int64(2126), 'tri': np.int64(2178), 'pound': np.int64(1641), 'pr

In [18]:
## Model Training with TF-IDF
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

spam_ham_detection_model_tfidf = GaussianNB()
## Training the model with training data
spam_ham_detection_model_tfidf.fit(X_train,y_train)
spam_ham_detection_model_tfidf

GaussianNB()

In [19]:
## Predicting Test data
y_pred = spam_ham_detection_model_tfidf.predict(X_test)
print(y_pred)

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0]


In [20]:
# Performance Matrix
from sklearn.metrics import accuracy_score,classification_report
print("Model Accuracy =" , accuracy_score(y_test,y_pred)*100)
print(classification_report(y_test,y_pred))

Model Accuracy = 90.0
              precision    recall  f1-score   support

           0       1.00      0.89      0.94        19
           1       0.33      1.00      0.50         1

    accuracy                           0.90        20
   macro avg       0.67      0.95      0.72        20
weighted avg       0.97      0.90      0.92        20

